![First Power Bi](.\images\dbachecks-logo.png )
# Setting up the containers for the rest of the dbachecks notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)


## Create the folders and the credential

Before you run this code, you will have to enter the directory that this notebook is saved in.   

This is a restriction of the Jupyter Notebook in Azure Data Studio as the PowerShell session does not know where it is locatBut I have kept it.d.  

The code below will create a directory called dbachecks-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the $FolderPath but you will have to do this in every notebook.

In [ ]:
# Alter the Notebook Directory below before running this code
$notebookDirectory ='C:\Users\mrrob\OneDrive\Documents\GitHub\JupyterNotebooks\notebooks\NotDotNet\dbachecks'
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Recurse -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Recurse -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
$dockercompose = (Get-Content $notebookDirectory\dockercompose.yml) -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content $FolderPath\docker-compose.yml



## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo).ve.

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run 

`docker pull sqldbawithabeard/dbachecksdemo`


In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
Set-Location $FolderPath
docker-compose up -d

If you get an error you might need to add the user you have shared your drives with Docker modify permissions to the \Documents\ directory in your user profile
More details https://docs.docker.com/docker-for-windows/ or troubleshoot in the normal way

Otherwise continue to the next notebook

If there are warnings above - Check the containers are running - Look at the status column

In [ ]:
docker ps -a

If there are warnings above and the containers are running - check the logs

In [ ]:
docker logs dbachecks_SQL2019_1

In [ ]:
docker logs dbachecks_SQL2019-1_1

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up